# H1 Title: How Useful Will Telemedicine Be?: Examining Hospital and Internet Access in Cook County
# H1 Abstract:
Telemedicine has become much more prominent in the recent past. However, little literature has been dedicated to understanding the spatial patterns in the Internet access necessary to have quality telehealth visits. This piece fills such a gap. This paper will detail the creation of an interactive dashboard using internet data from the American Community Survey and provider data from the NPI Registry maintained by Center for Medicaid Services. The dashboard shows that (in Cook County) significant clustering exists in lack of Internet connection (p<.01), no relationship exists between internet access and provider access (p=.81), with unclear evidence regarding the clustering patterns of providers. These findings show the need for additional internet connection in Cook County and suggest that future research would be useful for maintaining data quality on where hospitals exist in the county.


# H2 Introduction: 
    Since the onset of the pandemic, telemedicine use has proliferated. Telemedicine became one of the core tools to deliver medicine during that time (Colbert et al., 2020). Levels of telemedicine use persistently outpace pre-pandemic levels in multiple countries throughout the world (Mehrotra et al., 2021). Telemedicine appears more important now than ever, and it is not going away.
	However, internet access proves to be a barrier to the widespread diffusion and use of telemedicine in the United States (Quinton et al., 2021). Those without access to the internet have been shown to be less likely to use online medical records/online health portals (Trivedi et al., 2021; Bush et al., 2018). Rural households, which often get left behind in the race to connect to the Internet (Bell et al., 2023), also tend to be the ones that do not have any health insurance (Curtis et al.,2022 ). So, even though the innovation that has come as the result of the Internet should be lauded, it has highlighted socioeconomic disparities in access to care in new ways.
	Despite the issues with internet access, the definition of Medically Underserved Areas has not evolved. As long as the Internet is not deemed a “health care service,” (Commonwealth of Pennsylvania, 2023), this issue will continue to be neglected. Studying the distribution of Internet access in households vis a vis the distribution of hospitals can provide valuable insights regarding what areas might benefit most from telemedicine services and which communities might be left behind by both sets of opportunities. Because dashboards are interactive tools, they serve as a unique tool that could help healthcare providers, policy makers, insurance companies, and the lay public alike understand the relationships between these two variables.


# H2 Data and preprocessing: 
    Internet data were gathered from NHGIS (Manson et al., 2023). Internet access data were based on the American Community Survey in 2021. The data were joined between the shape file and the csv file. The original csv file contained more columns than I needed for this project. So, I filtered down the number of columns with which I was dealing to make sure that I could handle all of the data appropriately. I used GISJOIN to allow the shape and csv files to join, the state and county fields to filter appropriately, GEO_ID, which is unique for each ID, AOU2E001, which is the total number of people who responded to the survey, and AOU2E013, which is the number of people who responded to the survey who said that they had no Internet.  Data were then filtered down to Cook County census tracts only, and saved internally on my machine. With the file saved on my computer, I could minimize the amount of memory that my computer uses in the process of dealing with the file. I made a field entitled 'normal,' which shows the percentage of total folks who do not have Internet in a given census tract. The code to do this is below. 

In [ ]:
import pandas as pd
import geopandas as gpd

#illinois_internet_shape = gpd.read_file("C:/Users/natha/OneDrive/Documents/nhgis0010_shape/nhgis0010_shape/nhgis0010_shapefile_tl2021_us_tract_2021/US_tract_2021.shp")
#illinois_internet_csv = pd.read_csv("C:/Users/natha/nhgis0010_ds254_20215_tract_corrected.csv", usecols = ['GISJOIN', 'STATE', 'COUNTY','TRACTA', 'GEO_ID', 'AOU2E001', 'AOU2E013'],encoding='latin1')

#pd.read_csv(filename, usecols = [])
#illinois_internet = illinois_internet_shape.merge(illinois_internet_csv, on = 'GISJOIN')
#cookcountyinternet = illinois_internet[illinois_internet.STATE == 'Illinois']
#cookcountyinternet = cookcountyinternet[cookcountyinternet.COUNTY == 'Cook County']
#cookcountyinternet.to_file('C:\\Users\\natha\\OneDrive\\Documents\\cookcountyinternet.shp')
cookcountyinternet = gpd.read_file('C:\\Users\\natha\\OneDrive\\Documents\\cookcountyinternet.shp')
cookcountyinternet['normal'] = cookcountyinternet['AOU2E013']/cookcountyinternet['AOU2E001']

Provider data came from CMS. Specifically, provider data came from the NPI Registry, which keeps records on all care facilities that receive funding from the Center for Medicare Services (CMS, 2023). I filtered first down to providers just in the state of Illinois and save it to my machine to minimize the amount of memory the computer uses. The code to do this is below. 

In [ ]:
#providerdata = pd.read_csv("C:/Users/natha/OneDrive/Documents/npidata_pfile_20230306-20230312.csv", low_memory=False)
#providerdata = providerdata[providerdata['Provider Business Practice Location Address State Name'] == 'IL']
#providerdata.to_csv('C:\\Users\\natha\\OneDrive\\Documents\\providerdata.csv')

Once I got the code in the condensed format, I then compared all of the providers in the list to a list of zip codes that reside in Cook County. I got all of the providers' 5 digit zip code, and then used a comparison method isin() to get the final list of providers that are in the county. The code to do this is below:

In [ ]:
illinoiszipcodes = pd.read_excel("C:/Users/natha/OneDrive/Documents/zip codes in cook county.xlsx").astype(str)
providerdata = pd.read_csv('C:\\Users\\natha\\OneDrive\\Documents\\providerdata.csv')
providerdataslim = providerdata[['NPI', 'Entity Type Code', 'Provider Credential Text', 'Provider First Line Business Practice Location Address', 'Provider Second Line Business Practice Location Address', 'Provider Business Practice Location Address City Name', 'Provider Business Practice Location Address State Name', 'Provider Business Practice Location Address Postal Code']]
providerdataslim = providerdataslim.rename(columns={'Provider First Line Business Practice Location Address': 'address_line_one', 'Provider Second Line Business Practice Location Address': 'address_line_two', 'Provider Business Practice Location Address City Name': 'city', 'Provider Business Practice Location Address State Name': 'state', 'Provider Business Practice Location Address Postal Code': 'zip_code'})
idx = 0
providerdataslim['fivedigitzip'] = ''
s1 = slice(5)
while idx < len(providerdataslim):
    if len(str(providerdataslim['zip_code'][idx])) == 9:
        providerdataslim['fivedigitzip'][idx] = str(providerdataslim['zip_code'][idx])[s1]
    else:
        providerdataslim['fivedigitzip'][idx] = str(providerdataslim['zip_code'][idx])
    idx += 1
providers = providerdataslim[providerdataslim.fivedigitzip.isin(illinoiszipcodes.ZIP)]
print(providers)

The data on the providers, as I can see from the print statement, do not have the points in there, just the addresses. So, in order to plot these, I need to get the points. I used the nominatim geocoder to get these points. I saved this file internally, and then called it every time I needed it thereafter. The code to do this is as follows: (It's worth noting that not all locations got geocoded correctly. As a matter of fact, when I first geocoded with nominatim, I only got the correct address for about 70% of the addresses in the original providers list. While this is imperfect, it is the best I have as of now.)  The complete flow of data and preprocessing is illustrated in the attached figure: https://drive.google.com/file/d/1Iwo_WLSyAJd4lrjZ9QaOQIirNstaDYZs/view?usp=sharing.

In [ ]:
import fiona

providers['fullAddress'] = providers.address_line_one + ' ' + providers.city + ' ' + providers.state + ' ' + providers.fivedigitzip
    #listofsites = listofsites.append(jsonData)
#mydata = pd.DataFrame(data=listofsites)
#geocoded = gpd.tools.geocode(providers['fullAddress'],provider='nominatim', user_agent="test",timeout=None)
#geocoded.to_file('C:\\Users\\natha\\OneDrive\\Documents\\geocoded_table.csv')
#geocoded.to_file('C:\\Users\\natha\\OneDrive\\Documents\\geocoded_shape.shp')
geocoded = gpd.read_file('C:\\Users\\natha\\OneDrive\\Documents\\geocoded_shape.shp')
geocoded.head()

# H2 Methods:
To understand the relationship between internet and providers, I tried three different combinations of maps and tests: internet only, providers only, and combined. This way, I could have the strength of showing as much or as little data as I wanted in any given situation, which would especially be helpful if I wanted to show tihs to an outsider who may not know a lot of context about mapping or the issue at hand. however, in order to make these tests, I had to continue to prepare the data. 
    I started by trying to get the internet and the address of provider data in the same dataset so I could compare internet vs. providers in the same maps and tests. To start, I performed a spatial join on the internet and address data. Next, I grouped by and summarized the number of roviders in each census tract. Once I slimmed down the number of columns in my dataset, I was almost good to go at that point. However, one thing I noticed was that, at first, it seemed like the map and the regression told two different stories. After exploring the data further, I realized that, when I spatially joined the data, it created an observation for each of the providers. While that is helpful for getting point level geometry for the plots, it could screw up any future statistical tests because it would improerly weight certain observations over others. To mitigate this, i concluded by dropping all duplicates. Here is the code for that:
To test whether or not clustering exists between all of the internet data, I used spatial autocorrelatoin and the following hypotheses:
H0: No relationship exists in the distribution of internet access. Ha: A spatial relationship exists for internet access across Cook County.
To test whether or not clustering exists between all of the provider data, I used spatial autocorrelatoin and the following hypotheses:
H0: No relationship exists in the distribution of providers. Ha: A spatial relationship exists for providers in Cook County.
To test if a relationship exists between providers and internet access, I will use the following hypotheses:
Hypotheses: H0: No relationship between providers and internet access at census tract level. Ha: Relationship exists between providers and internet access census tract level.

In [ ]:
from geopandas import GeoDataFrame

total = cookcountyinternet.sjoin(geocode, how = 'left', predicate = 'contains')
cookcountyinternet.head()
total2 = total.groupby('GEOID').count()
total2 = total2[['address']].rename(columns = {'address':'count'})
mergedframe = total2.merge(total, how = 'left', left_on='GEOID', right_on='GEOID')
mergedframe = mergedframe[['GEOID', 'count', 'NAMELSAD', 'AOU2E001', 'AOU2E013', 'geometry', 'normal']]
mergedframegeo = GeoDataFrame(mergedframe, crs='EPSG:4326', geometry='geometry')
idx = 0
mergedframe = mergedframe.drop_duplicates()

Next, I had to get the data ready for a linear regression model. Linear regression models require a few more steps in Python. For starters, the functions that perform these tests primarily take data in arrays, so I had to convert everything to an array. Then, once I got to the end of this process, I noticed that a lot of the census tracts had zero providers in them. Hence, to make sure that any results I got from the regression were not because of some correlation between the error term and the outcome variable, I used robust standard errors. Here is the code I used for that:

In [ ]:
import numpy as np
import statsmodels.api as sm
frameforreg = mergedframe[['normal', 'count']]
frameforreg = frameforreg[~frameforreg.normal.isnull()]

arraysforreg = frameforreg.to_numpy()

normarray = np.zeros(shape=len(arraysforreg))
countarray = np.zeros(shape=len(arraysforreg))
idx = 0
while idx < len(normarray):
    normarray[idx] = arraysforreg[idx][0]
    idx +=1
idx = 0
while idx < len(countarray):
    countarray[idx] = arraysforreg[idx][1]
    idx += 1
y = countarray
x = normarray
x = sm.add_constant(x)
model = sm.OLS(y,x)
results = model.fit(cov_type = 'HC3')

results, the variable that holds the results of the regression, can get called later in the dashboard. Finally, I had to prepare the data for spatial autocorrelation on both internet access and providers. Now, in order to do that, I needed to weight each observation appropriately based on the number of counties around it. I used rook contiguity to do this because I thought it would be the most simple definition with the least number of neighbors generated, which I thought would make the data analysis easier with so many census tracts inside Cook County. Here is the code I used to prepare the spatial autocorrelation for the internet and providers respecitvely: 

In [ ]:
from splot.esda import moran_scatterplot,lisa_cluster
from pysal.explore import esda
import matplotlib.pyplot as plt

cookcountyinternet_ac = cookcountyinternet[~cookcountyinternet.normal.isnull()]
rook_internet = weights.Rook.from_dataframe(cookcountyinternet_ac,idVariable="GISJOIN")
rook_internet.transform = 'R'
morans = esda.Moran(cookcountyinternet_ac['normal'], rook_internet)
moran_scatterplot(morans);
moransIinternet = morans.I
moransIinternetpvalue = morans.p_sim


mergedframegeo_ac = mergedframegeo.drop_duplicates()
rook_provider = weights.Rook.from_dataframe(mergedframegeo_ac,idVariable="GEOID")
rook_provider.transform = 'R'
morans1 = esda.Moran(mergedframegeo_ac['count'], rook_provider)
moran_scatterplot(morans1);
moransIprovider = morans1.I
moransIproviderpvalue = morans1.p_sim


I will note that I had to install pysal, a package that creates the scatter plot, before I could move any further. Now, with all of this in mind, I could use the rest of my code to start the dashboard. With this dashboard, I was interested in three sets of hypotheses:
H0: No relationship exists between the proportion of people who have no internet and the number of healthcare providers in that area. Ha: A relationship exists between the proportion of people who have no Internet and the number of healthcare providers in that area.
Ha: No clustering exists in the spatial distribution of healthcare providers across Cook County. Ha: Clustering exists in the spatial distribution of healthcare providers across Cook County.
Ha: No clustering exists in the spatial distribution of those who have no Internet access across Cook County. Ha: Clustering exists in the spatial distribution of those who have no internet access across Cook County.
Once I defined all of these hypotheses and did all of my prep work, I was then ready to start coding the dashboard. Here is the code that creates the dashboard:

In [ ]:
import ipywidgets as widgets
from IPython.display import display
#ax = cookcountyinternet.plot('normal', cmap = 'RdYlGn_r', scheme='natural_breaks')
def selection(change):
    if change['new'] == 'Internet':
        cookcountyinternet.plot('AOU2E013', cmap = 'RdYlGn_r', scheme='natural_breaks', legend = True)
    if change['new'] == 'Internet Normalized by Population':
        cookcountyinternet.plot('normal', cmap = 'RdYlGn_r', scheme='natural_breaks', legend = True)
    if change['new'] == 'Internet and Providers':
        ax = cookcountyinternet.plot('normal', cmap = 'RdYlGn_r', scheme='natural_breaks', legend = True)
        geocode[~geocode.address.isnull()].plot(marker='o', color='blue', markersize = 1, ax = ax, legend = True)
    if change['new'] == 'Internet Spatial Autocorrelation':
        moran_scatterplot(morans);
        print('Morans I = ' + str(moransIinternet))
        print('p-value = ' + str(moransIinternetpvalue))
    if change['new'] == 'Providers Spatial Autocorrelation':
        moran_scatterplot(morans1);
        print('Morans I = ' + str(moransIprovider))
        print('p-value = ' + str(moransIproviderpvalue))
    if change['new'] == 'Providers':
        mergedframegeo.plot('count', cmap = 'Reds', scheme = 'natural_breaks', legend = True)
    if change['new'] == 'Regression Results':
        print(results.summary())
        

final = widgets.RadioButtons(
    options=['Internet and Providers', 'Internet Normalized by Population', 'Internet', 'Internet Spatial Autocorrelation', 'Providers', 'Providers Spatial Autocorrelation', 'Regression Results'],
    value=None,
    layout={'width': 'max-content'},
    description='Display:',
    disabled=False
)
final.observe(selection,names='value')
final

The dashboard contains seven radio buttons, each of which are mutually exclusive. When one clicks on one of the buttons, it prints the results of the corresponding operation. The maps sometimes take a second to load, but they show up eventually. The autocorrelation options show the moran plots for the respective description, in addition to the Moran's I value and the p value for the I value. The regression shows the results of the linear regression with robust standard errors described earlier. All outputs show up in the log, which keeps a running list of the results.

# H2 Results and Discussion:
(To see the results of the dashboard, please check out the pictures in this folder: https://drive.google.com/drive/folders/1fvsk3IXhQfkbxUHldm3a6OSscujalpkO?usp=sharing)
#H3 Initial Visual Insights
From the initial visuals, one can see a number of things. First, one can see that providers seem to be slightly clustered as people move in toward the city. The same can be seen in both internet maps. As people move in toward the city, they tend to have less internet. While such a pattern is obvious for both non-normalized and normalized by population graphs, this is especially obvious in the graph that is normalized by population. 
# H3 Linear Regression
The linear regression yields a beta coefficient of 0.11, meaning that a 1 unit increase in "normal" (i.e. as the proportion of people in a census tract who have no internet goes from 0 to 100%), corresponds to a .11 provider increase in the community. The p value of .81 shows that a value of this magnitude or greater could be expected by chance 81% of the time. Because this value is less than .05, the relationship between the proportion of people who do not have internet and providers in the community is not statistically significant. Given that internet access has been increasingly recognized as a social determinant of health (Benda et al., 2020), the finding appears somewhat surprising. The findings suggest that applications of social determinants of health must be considered context specific and do not necessarily apply in all contexts.
# H3 Spatial Autocorrelation
The Moran's I for the internet access is 0.48 with a p value of .001, meaning that correlation of this magnitude or greater would happen by chance less than 1% of the time. Since this is less than .05, the null hypothesis can be rejected. Significant clustering exists amongst internet access in Cook County. Given that Cook County's Community Health Status Assessment reports disparities in internet access by part of the county, this finding is not particularly surprising (CCDPH, 2022, p. 51). The p value of the Moran's I for the healthcare providers is .022, meaning that we could expect clustering to this degree or greater by chance around 2% of the time. While this is statistically significant, it seems like a couple of outliers might have had a significant impact in this result. The legend of the provider map and the scatterplot of the Moran's I for providers suggest that a small handful of outliers might be significantly tainting this result. With additional information on the rest of the providers from more geocoding services, a more holistic picture of spatial autocorrelation amongst the providers could be gleaned.
# H3 Strengths
    This project has two main strengths: the granularity with which you can analyze patterns and the scope of the project. The dashboard aspect of the projects allows for users to have the freedom to receive information in whichever way they understand it best, while not compromising the quality of the data. Not only does the project have high quality data from a reputable source that collects this information on a regular basis (CMS, 2023), it also has a high quantity of data as well. As stated earlier, Cook County is the second largest county by population, so the dashboard shows that maps, spatial autocorrelation, and regressions can help synthesize a large amount of data.
# H3 Limitations
    This project is not perfect and, as such, has some notable limitations. These include: geocoder and lack of clarity on what types of providers exist here. Like I said earlier, the geocoder only caught roughly 70% of the providers form the initial data. Because I do not know which ones got excluded, I have no way of ensuring that the ones that got included are representative of the whole. The geocoder might have introduced some selection bias here. Also, in this study, I make no distinction between the kinds of providers that I am mapping. I partly did this just to make sure I had enough data. As discussed earlier, many census tracts did not even have any providers. That being said, though, some in the data set are individuals versus others are organizations, and they represent a wide variety of subspecialties (CMS, 2023). All of these might have varying interest in adapting telemedicine going forward. So, in order to understand the gaps necessary to fill, more context would be needed on the provider side.
# H3 Future Directions
    This project is not comprehensive and, as such, invites future work in a number of directions. First, Cook County, although large, is just one county in the United States. Future dashboards could be built to look at other counties or states across America with respect to internet accessibility and hospital distribution. Also, even within Cook County, more directions of investigation still exist. Given that income and education seem to be determinants of internet access on a broader scale (Chaudhuri et al., 2005), one could map income and education on to similar plots to observe potential patterns and/or effect modifiers. Finally, more formal analysis could look at LISA clusters. LISA clusters show more specifically where the clusters in any given map are and how different on each axis a given census tract is from its neighbor (Ajayakumar, 2023). While the maps communicate a lot of information quickly about the distribution of providers and internet access, LISA statistics could allow for more granular analysis. 
    Indpeendent of new research ideas, the current project could be refined by trying it with new sources of data. For example, many different geocoders exist (List of Online Geocoding Services, 2013). Future researchers could try to repeat this dashboard for Cook county while also trying to use different geocoders to see if gaps in provider data could be filled by the other services. Also, IPUMS releases new data from the American Community Survey for each year (Manson et al., 2023). Future researchers could download future iterations of American Community Survey data and provider data to see if patterns change. Also, they could download historical data, put a date slider on this dashboard, and track how the relationship between Internet access and hospitals changed over time. The potential for innovation with this dashboard remains plentiful.

# H2 Conclusion:
Internet access and healthcare providers appear to be clustered in Cook County, although the clustering in internet access appears much more clear with respect to Internet access. The lack of internet access does not appear to be correlated with where healthcare providers are located in Cook County. These findings provide a unique understanding of the context with respect to access to care in Cook County, especially as it pertains to telemedicine access moving forward. These findings show the need for more internet access in certain communities, and further research could consider illuminating where clusters of poor internet access are.

#H2 References:
Ajayakumar, J. (2023) Spatial Autocorrelation. Case Western Reserve University, PQHS 427.
Bell, N., Hung, P., Lòpez-De Fede, A., & Adams, S. A. (2023). Broadband access within Medically Underserved Areas and its implication for telehealth utilization. The Journal of Rural Health, n/a(n/a). https://doi.org/10.1111/jrh.12738
Benda, N. C., Veinot, T. C., Sieck, C. J., & Ancker, J. S. (2020). Broadband Internet Access Is a Social Determinant of Health! American Journal of Public Health, 110(8), 1123–1125. https://doi.org/10.2105/AJPH.2020.305784
Bush, R. A., Barlow, H., Pérez, A., Vazquez, B., Mack, J., & Connelly, C. D. (2018). Internet Access Influences Community Clinic Portal Use. Health Equity, 2(1), 161–166. https://doi.org/10.1089/heq.2018.0019
Chaudhuri, A., Flamm, K. S., & Horrigan, J. (2005). An analysis of the determinants of internet access. Telecommunications Policy, 29(9), 731–755. https://doi.org/10.1016/j.telpol.2005.07.001
CMS. (2023). NPPES NPI Registry. Retrieved April 1, 2023, from 
	https://npiregistry.cms.hhs.gov/search. 
Cook County Department of Public Health (2022). Community Health Status Assessment: Appendix D. chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/https://cookcountypublichealth.org/wp-content/uploads/2022/06/CHSA_appendix-D_final.pdf
Curtis, M. E., Clingan, S. E., Guo, H., Zhu, Y., Mooney, L. J., & Hser, Y.-I. (2022). Disparities in digital access among American rural and urban households and implications for telemedicine-based services. The Journal of Rural Health, 38(3), 512–518. https://doi.org/10.1111/jrh.12614
Colbert, G. B., Venegas-Vera, A. V., & Lerma, E. V. (2020). Utility of telemedicine in the COVID-19 era. Reviews in Cardiovascular Medicine, 21(4), Article 4. https://doi.org/10.31083/j.rcm.2020.04.188
Commonwealth of Pennsylvania. (2023). Underserved Areas. Department of Health. https://www.health.pa.gov:443/topics/programs/Primary%20Care/Pages/Underserved-Areas.aspx
List of Online Geocoding Systems. (2013, July 1). TAMU GeoServices. https://geoservices.tamu.edu/Services/Geocode/OtherGeocoders/default.aspx
Steven Manson, Jonathan Schroeder, David Van Riper, Tracy Kugler, and Steven Ruggles. IPUMS National Historical Geographic Information System: Version 17.0 [dataset]. Minneapolis, MN: IPUMS. 2022. http://doi.org/10.18128/D050.V17.0
Mehrotra, A., Bhatia, R. S., & Snoswell, C. L. (2021). Paying for Telemedicine After the Pandemic. JAMA, 325(5), 431–432. https://doi.org/10.1001/jama.2020.25706
Neha Trivedi, P., Patel, V., Johnson, C., & Wen-Ying Sylvia Chou, P. (2021). Barriers to Accessing Online Medical Records in the United States. 27. https://www.ajmc.com/view/barriers-to-accessing-online-medical-records-in-the-united-states

Quinton, J. K., Ong, M. K., Vangala, S., Tetleton-Burns, A., Webb, A., Sarkisian, C., Casillas, A., Kakani, P., Han, M., & Pirtle, C. J. (2021). The Association of Broadband Internet Access and Telemedicine Utilization in rural Western Tennessee: An observational study. BMC Health Services Research, 21(1), Article 1. https://doi.org/10.1186/s12913-021-06746-0